In [241]:
!pip3 install openpyxl
import pandas as pd
import re


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


In [242]:
df = pd.read_excel(r'C:\Users\john_\Desktop\UN-project\UPR_cycle2.xlsx')

In [243]:
df.index+=1
df.head(2)

,Text,Countries Concerned,Reccomending Body,Document Symbol,Themes,Affected Persons,Sdgs,Document Publication Date,UPR Reccomending States,UPR Position,Type,OHCHR Annotation Id,UPR Session,Regions Concerned,Recommending Regions,Date of publication on UHRI
1,117.22 Promptly ratify the Optional Protocol t...,- Uganda,- UPR,A/HRC/34/10,- Labour rights and right to work\n- Ratificat...,- Indigenous peoples,NaN,2016-12-27,- Guatemala,- Noted,- Recommendations,4100eed4-fcf7-42d6-941a-a89493e77e6c,- 26th Session - Cycle 2,- Africa,- Latin America and the Caribbean,2018-06-20 13:50:21.503
2,133.247 Ensure full and equal access to modern...,- Venezuela (Bolivarian Republic of),- UPR,A/HRC/34/6,- Sexual & reproductive health and rights,- Indigenous peoples\n- Persons living in rura...,NaN,2016-12-27,- Denmark,- Supported,- Recommendations,f910ce5d-0d1c-45de-b64b-3050df7679dc,- 26th Session - Cycle 2,- Latin America and the Caribbean,- Western Europe & Others,2018-05-04 12:18:58.713


# Data Cleaning

##### - Check unique number of text & annotation id to establish primary key
##### - Both have 625 unique entries which is the same as the total database entries
##### - Normalise by removing one of these, two identifiers is uneccesary we are not gaining anymore information

In [244]:
text_count = df.Text.unique().size
print(text_count)

625


In [245]:
annotation_id_count = df["OHCHR Annotation Id"].unique().size
print(annotation_id_count)

625


##### Columns to be dropped
##### Annotation ID, Recommending body, Text, Affected persons, Sdgs, Document publication date, session, date
##### check all entries are recommendation by seeing if text starts with number, then drop this column. 

In [246]:
drop_columns = ["OHCHR Annotation Id","Reccomending Body", "Document Publication Date", "Sdgs", "UPR Session", "Date of publication on UHRI", "Affected Persons"]

In [247]:
df.drop(drop_columns, inplace=True, axis=1)

In [248]:
df.head(2)

,Text,Countries Concerned,Document Symbol,Themes,UPR Reccomending States,UPR Position,Type,Regions Concerned,Recommending Regions
1,117.22 Promptly ratify the Optional Protocol t...,- Uganda,A/HRC/34/10,- Labour rights and right to work\n- Ratificat...,- Guatemala,- Noted,- Recommendations,- Africa,- Latin America and the Caribbean
2,133.247 Ensure full and equal access to modern...,- Venezuela (Bolivarian Republic of),A/HRC/34/6,- Sexual & reproductive health and rights,- Denmark,- Supported,- Recommendations,- Latin America and the Caribbean,- Western Europe & Others


In [249]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [250]:
df = df[df['Text'].apply(lambda x: has_numbers(x))]
df.drop(['Text', 'Type'], inplace=True, axis=1)

In [251]:
df.index = df.index.set_names(['Recommendation_id'])
df.head(2)

,Countries Concerned,Document Symbol,Themes,UPR Reccomending States,UPR Position,Regions Concerned,Recommending Regions
Recommendation_id,,,,,,,
1,- Uganda,A/HRC/34/10,- Labour rights and right to work\n- Ratificat...,- Guatemala,- Noted,- Africa,- Latin America and the Caribbean
2,- Venezuela (Bolivarian Republic of),A/HRC/34/6,- Sexual & reproductive health and rights,- Denmark,- Supported,- Latin America and the Caribbean,- Western Europe & Others


#### Standardize the naming convention, by removing hyphens and spaces, of all data entries.

In [252]:
def remove_leading_char(data_entry):
    """"Clean the naming convention for data entries of all columns"""
    clean_aff = re.sub(r'^[^A-Z]*', ' ', data_entry)
    if clean_aff:
        return clean_aff

In [253]:
df2 = df["UPR Reccomending States"].apply(lambda x: remove_leading_char(x))

TypeError: expected string or bytes-like object

In [ ]:
df2

In [ ]:
theme_matrix = {'ratification': ['tag1', 'tag2'], 'next_theme':'tag1'}
theme_matrix['ratification']